In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import change_X_data_1, change_y_data, change_X_data_2, create_y_test_file



In [ ]:
X = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_train.csv")
y = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/y_train.csv")

In [ ]:
X2 = change_X_data_2(X)


In [ ]:
X2.head()

In [ ]:
# Access value in row 2 (index), column 'column_name'
value = X2.at[2, 'column 1']  # or df.loc[2, 'column_name']
print(value)


In [ ]:
X2.info()

In [ ]:
X3 = np.array([np.array(row) for row in X2["column 1"]])

In [ ]:
X3[0][1]

In [ ]:
X3.shape

In [ ]:
#y = change_y_data(y)
y1 = y.drop(columns=["obs_id"])

y1.head()

In [ ]:
y1.info()

In [ ]:
y2 = np.array(y1["eqt_code_cat"])
y2.shape

In [ ]:
print(np.any(np.isnan(X3)))  # Check features for NaNs
print(np.any(np.isnan(y2)))

In [ ]:
X3 = np.nan_to_num(X3)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X3, y2))

In [ ]:
for x, y in dataset.take(3):  # Use `.take()` to limit output to 3 samples
    print("Features:", x.numpy())
    print("Label:", y.numpy())


In [ ]:
print(dataset.element_spec)


In [ ]:
batch_size = 32
dataset = dataset.shuffle(buffer_size=len(X3))  # Shuffling the dataset



In [ ]:
dataset = dataset.batch(batch_size)

In [ ]:
for x, y in dataset.take(1):
    print(f"Features batch shape: {x.shape}, Labels batch shape: {y.shape}")

In [ ]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [   GRU(64, return_sequences=True, input_shape = (100, 11) ),            
        Dropout(0.2),
        GRU(64),
        Dropout(0.2),
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(24, activation = 'softmax',   name = "output")
        
        
    ], name = "first_multiclass_model" 
)

In [136]:
model.summary()

Model: "first_multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_8 (GRU)                     │ (None, 100, 64)        │        14,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 24)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,850 (499.42 KB)

 Trainable params: 42,616 (166.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 85,234 (332.95 KB)

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history = model.fit(
    dataset,
    epochs=50,
)

In [ ]:
model.save("../models/model_matrix_3.h5")  # Saves the model as an HDF5 file


In [ ]:
# test_loss, test_accuracy = model.evaluate(dataset)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# # Reshape it to (1, 100, 11) to match the model input shape
# single_sequence = np.expand_dims(x_test_3[0], axis=0)

# # Make a prediction with the trained model
# prediction = model.predict(single_sequence)
# yhat = np.argmax(prediction)

# print("Prediction for the single sequence:", yhat)


In [ ]:
# n = 14

# prediction = model.predict(np.array(x_test_3.iloc[n]).reshape(1,12))
# prediction_p = tf.nn.softmax(prediction)
# yhat = np.argmax(prediction_p)
# print(prediction_p)
# print("prediction :")
# print(yhat)
# print("expected value : ")
# print(y.iloc[n])

In [ ]:
# m = 1000
# S = 0

# for i in range(m):
    
#     prediction = model.predict(np.array(x_test.iloc[i]).reshape(1,12))
#     prediction_p = tf.nn.softmax(prediction)
#     yhat = np.argmax(prediction_p)

#     if yhat == y["eqt_code_cat"].iloc[i] :
#         S+=1
    
# print(f"{(S/m)*100}% of precision")

    